# Search and Summarize News

In [1]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython import display  # for pretty printing
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import openai  # for using GPT and getting embeddings
import os  # for loading environment variables
import requests  # for making the API requests
from tqdm.notebook import tqdm  # for printing progress bars

In [2]:
# Load environment variables
news_api_key = os.getenv("NEWS_API_KEY")

GPT_MODEL = "gpt-3.5-turbo"

# User asks a question
USER_QUESTION = "How is Putin doing after the coup attempt?"

# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input: list[str]) -> list[list[str]]:
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]

QUERIES_INPUT = f"""
You have access to a search API that returns recent news articles.
Generate an array of search queries that are relevant to this question.
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms.
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2'].
Be creative. 
Please be concise and include only the keywords in the queries. 
The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

print(queries)

# Let's include the original question as well for good measure
queries.append(USER_QUESTION)

def search_news(
    query: str,
    news_api_key: str = news_api_key,
    num_articles: int = 50,
    from_datetime: str = "2023-06-21", 
    to_datetime: str = "2023-07-15",
) -> dict:
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    result = search_news(query)
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

# remove duplicates
articles = list({article["url"]: article for article in articles}.values())

HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer which will be used to rank search results. 
Pretend you have all the information you need to answer, but don't use any actual facts. Instead, use placeholders
like NAME did something, or NAME said something at PLACE. Also pretend you are in the year 2030 and have access to all the knowledge up to that year. 

User question: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
article_embeddings = embeddings(
    [
        f"{article['title']} {article['description']} {article['content'][0:100]}"
        for article in articles
    ]
)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding))
    

scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. Reference the relevant search result urls as markdown links.
"""

completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=0.5,
    stream=True,
)


text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))

Putin's regime has been profoundly shaken after the coup attempt by the Wagner mercenary company and its leader, Yevgeny Prigozhin. While Putin has tried to show that he is firmly in control, Russia's elite is unsure of his actual level of control. The Kremlin has released footage of Putin being greeted by adoring crowds in the street and attending a tech exhibition for Russian companies in Moscow, but there is ongoing debate about the stability of his rule. Some experts argue that Putin's rule was never truly threatened, but Putin's actions suggest otherwise. 

To learn more about Putin's struggles to recover from the coup attempt, you can read the following articles:

1. [Putin’s regime struggles to recover from coup attempt, says political analyst](https://news.yahoo.com/putin-regime-struggles-recover-coup-134000506.html)
2. [Vladimir Putin Claims He's Back in Control. Russia's Elite Isn't Sure](https://www.ndtv.com/world-news/vladimir-putin-claims-hes-back-in-control-russias-elite-isnt-sure-4165442)
3. [What experts make of Putin’s rare public appearances after failed coup](https://www.semafor.com/article/06/29/2023/experts-putin-rare-public-appearances)
4. [After the rebellion, how well will Putin carry on his rule?](https://news.yahoo.com/rebellion-well-putin-carry-rule-174700716.html)
5. [The Russian coup-that-wasn’t fell flat. The next one might not | Rajan Menon and Daniel R DePetris](https://www.theguardian.com/global/commentisfree/2023/jun/28/russian-coup-vladimir-putin-wagner-ukraine-war)

In [3]:
hypothetical_answer

"After the coup attempt, Putin managed to successfully regain control of the government. He implemented several reforms to stabilize the country and strengthen his position. Putin's popularity soared as he was seen as a strong leader who protected the nation's interests. He continued to prioritize economic growth and modernization, leading to significant advancements in various sectors. Overall, Putin's leadership after the coup attempt has been widely regarded as successful and has solidified his position as a dominant figure in Russian politics."